# Attitude Dynamics and Control of a Nano-Satellite Orbiting Mars

In [1]:
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt

from pathlib import Path

import sys
sys.path.insert(0, r"../../")
import AttitudeKinematicsLib as ak

In [2]:
print("Contents of AttitudeKinematicsLib:")
for name in sorted(dir(ak)):
    if not name.startswith("_"):
        print(name)

Contents of AttitudeKinematicsLib:
BInvmat_CRP
BInvmat_EP
BInvmat_Euler
BInvmat_MRP
BInvmat_PRV
Bmat_CRP
Bmat_EP
Bmat_Euler
Bmat_MRP
Bmat_PRV
CRP
CRP_to_DCM
DCM_to_CRP
DCM_to_EP
DCM_to_Euler
DCM_to_MRP
DCM_to_PRV
DCM_utils
EP_to_DCM
EulerAngles
EulerRodriguesParameters
Euler_to_DCM
MRP
MRP_to_DCM
PRV
PRV_to_DCM
integrate_quaternion
normalize_quat
np
quat_derivative
quat_diff
quat_inv
quat_mult
rotation_matrix_x
rotation_matrix_y
rotation_matrix_z
skew_symmetric
solve_ivp
validate_DCM
validate_vec3
validate_vec4


# Module 1 - Practice Assignment

In [3]:
OUTPUT_DIR = Path("outputs")
OUTPUT_DIR.mkdir(exist_ok=True)

In [4]:
def export_txt(filename_stem: str, text_data: str) -> None:
    """
    Export mission result text to the outputs directory.

    Parameters
    ----------
    filename_stem : str
        Name of the output file without extension.
    text_data : str
        Text content to be written to the file.
    """
    file_path = OUTPUT_DIR / f"{filename_stem}.txt"
    file_path.write_text(text_data, encoding="utf-8")
    print(f"[Saved] {file_path}")


In [5]:
part1 = " ".join(str(x) for x in range(1, 10, 2))
print(part1)

export_txt(filename_stem="module1_part1", text_data=part1)

1 3 5 7 9
[Saved] outputs/module1_part1.txt


In [6]:
A = [[1, 2], 
     [9, 10]]

part2 = " ".join(str(A[i][j]) for i in range(len(A)) for j in range(len(A[0])))
print(part2)

export_txt(filename_stem="module1_part2", text_data=part2)

1 2 9 10
[Saved] outputs/module1_part2.txt


In [7]:
part3 = f"{np.pi:.5f}"
print(part3)

export_txt(filename_stem="module1_part3", text_data=part3)

3.14159
[Saved] outputs/module1_part3.txt


# Module 2 - Orbits

# Module 3 - Reference Frame Orientation

# Module 4 - Attitude Evaluation and Simulator

# Module 5 - Final Mission Assessment